In [ ]:
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# 1. Generate Random Boxes
def generate_boxes(n=50, min_s=5, max_s=15):
    boxes = []
    for i in range(n):
        w = random.randint(min_s, max_s) # x
        d = random.randint(min_s, max_s) # y
        h = random.randint(min_s, max_s) # z
        boxes.append({'id': i, 'w': w, 'd': d, 'h': h})
    return boxes

boxes = generate_boxes(60)

## Implementation: 3D Layer Packer

In [ ]:
class Packer3D:
    def __init__(self, bin_w, bin_d, bin_h):
        self.bin_w = bin_w
        self.bin_d = bin_d
        self.bin_h = bin_h
        self.packed_items = []
        
    def pack(self, items):
        # Sort by Height (Z) primarily, then Depth (Y)
        sorted_items = sorted(items, key=lambda x: (x['h'], x['d']), reverse=True)
        
        current_z = 0
        
        while sorted_items and current_z < self.bin_h:
            # Start a new Layer
            layer_items = []
            layer_h = 0
            current_y = 0
            
            # While we have space in Y (Depth)
            while sorted_items and current_y < self.bin_d:
                # Start a new Row
                row_items = []
                row_d = 0
                current_x = 0
                
                # Find items that fit in this row
                # We iterate through a copy or index to find fits
                remaining_items = []
                
                for item in sorted_items:
                    # Check if item fits in X
                    if current_x + item['w'] <= self.bin_w:
                        # Check if item fits in Y (initially any item fits new row, but subsequent rows are constrained)
                        if current_y + item['d'] <= self.bin_d:
                            # Check if item fits in Z
                            if current_z + item['h'] <= self.bin_h:
                                # Place Item
                                item['x'] = current_x
                                item['y'] = current_y
                                item['z'] = current_z
                                
                                self.packed_items.append(item)
                                row_items.append(item)
                                
                                current_x += item['w']
                                row_d = max(row_d, item['d'])
                                layer_h = max(layer_h, item['h'])
                            else:
                                remaining_items.append(item)
                        else:
                            remaining_items.append(item)
                    else:
                        remaining_items.append(item)
                
                # If we placed nothing in this row, break (layer is full in Y or no items fit)
                if not row_items:
                    break
                    
                # Update Y for next row
                current_y += row_d
                
                # Update list of items to pack (remove placed ones)
                # Note: The loop above is a bit simplified greedy. 
                # A proper implementation would remove items carefully.
                # Here we just replaced sorted_items with remaining_items which works for this simple logic
                sorted_items = remaining_items
            
            # If we placed nothing in this layer, break (bin full in Z)
            if not layer_items and not row_items: # row_items check handles the last row logic
                 break
                 
            # Update Z for next layer
            current_z += layer_h
            
        return self.packed_items

# Run
bin_size = 50
packer = Packer3D(bin_size, bin_size, bin_size)
packed = packer.pack(boxes)

print(f"Packed {len(packed)} / {len(boxes)} boxes.")

## Visualization (Matplotlib 3D)

In [ ]:
def plot_3d_packing(packed_items, bin_w, bin_d, bin_h):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Draw Bin Wireframe
    # Corners of the bin
    corners = np.array([
        [0, 0, 0], [bin_w, 0, 0], [bin_w, bin_d, 0], [0, bin_d, 0],
        [0, 0, bin_h], [bin_w, 0, bin_h], [bin_w, bin_d, bin_h], [0, bin_d, bin_h]
    ])
    # Edges
    edges = [
        [0,1], [1,2], [2,3], [3,0], # Bottom
        [4,5], [5,6], [6,7], [7,4], # Top
        [0,4], [1,5], [2,6], [3,7]  # Sides
    ]
    for edge in edges:
        ax.plot3D(*zip(*corners[edge]), color='black', linewidth=1)
        
    # Draw Boxes
    # We use voxels or bar3d. bar3d is easier for arbitrary sizes.
    colors = plt.cm.get_cmap('tab20', len(packed_items))
    
    for i, item in enumerate(packed_items):
        ax.bar3d(item['x'], item['y'], item['z'], 
                 item['w'], item['d'], item['h'], 
                 color=colors(i), alpha=0.8, edgecolor='k', linewidth=0.5)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f"3D Packing - {len(packed_items)} items")
    
    # Set limits
    ax.set_xlim(0, bin_w)
    ax.set_ylim(0, bin_d)
    ax.set_zlim(0, bin_h)
    
    plt.show()

plot_3d_packing(packed, bin_size, bin_size, bin_size)